# Placement Tokens Demo

This notebook demonstrates placing a job on an AP, using a Placement Token for authorization.

Use Shift+Enter to run the cells in this notebook.

Run the following cell to perform some initial setup for the notebook.  A button will be created that will allow you to request the Placement Token.  You will use your NetID to request the token.

In [1]:
import demo
demo.setup()

Button(button_style='primary', description='Request Token', layout=Layout(width='auto'), style=ButtonStyle())

HTML(value='')

HTML(value='')

<br><br>
Once you have obtained the token, create the object named `ap` with which you will interact with the Access Point.

In [ ]:
ap = demo.AP()

Print out the capabilities of your token.  You must be able to both inspect jobs and place jobs to use the rest of this notebook.  If your token does not allow you to do that, you might need to get another one.

In [ ]:
ap.describe_token()

List the jobs that have been placed on the AP. There may already be jobs from other users or previous runs.

In [ ]:
ap.show_job_count()

## Placing a job

Create the `job_description` object, which will hold the submit description of a test job that you will place.
This test job will sleep for 30 seconds then add a message with the date.

In [ ]:
job_description = demo.load_job_description("test_job.submit")

Place the job. The resulting job ID will be printed.

In [ ]:
placement = ap.place(job_description)
placement.show_job_ids()

Wait for the job to complete.

In [ ]:
placement.monitor_jobs(minutes = 5)

If the job is completed, run the following to copy the results back to the current directory.  The file `test_job_result.txt` will be created in the `work/` subdirectory.

In [ ]:
placement.retrieve_results()

The following will print the result of the job.

In [ ]:
print(open("work/test_job_result.txt", "r").read())